# Making Pandas DataFrames from API Requests
In this example, we will use the U.S. Geological Survey's API to grab a JSON object of earthquake data and convert it to a `pandas.DataFrame`.

USGS API: https://earthquake.usgs.gov/fdsnws/event/1/

### Get Data from API

In [1]:
import datetime as dt
import pandas as pd
import requests

yesterday = dt.date.today() - dt.timedelta(days=1)
api = 'https://earthquake.usgs.gov/fdsnws/event/1/query'
payload = {
    'format': 'geojson',
    'starttime': yesterday - dt.timedelta(days=30),
    'endtime': yesterday
}
response = requests.get(api, params=payload)

# let's make sure the request was OK
response.status_code

200

Response of 200 means OK, so we can pull the data out of the result. Since we asked the API for a JSON payload, we can extract it from the response with the `json()` method.

### Isolate the Data from the JSON Response
We need to check the structures of the response data to know where our data is.

In [2]:
earthquake_json = response.json()
earthquake_json.keys()

dict_keys(['type', 'metadata', 'features', 'bbox'])

The USGS API provides information about our request in the `metadata` key. Note that your result will be different, regardless of the date range you chose, because the API includes a timestamp for when the data was pulled:

In [3]:
earthquake_json['metadata']

{'generated': 1630426874000,
 'url': 'https://earthquake.usgs.gov/fdsnws/event/1/query?format=geojson&starttime=2021-07-31&endtime=2021-08-30',
 'title': 'USGS Earthquakes',
 'status': 200,
 'api': '1.12.1',
 'count': 14319}

Each element in the JSON array `features` is a row of data for our dataframe.

In [4]:
type(earthquake_json['features'])

list

Your data will be different depending on the date you run this.

In [5]:
earthquake_json['features'][0]

{'type': 'Feature',
 'properties': {'mag': 0.99,
  'place': '8 km SW of Volcano, Hawaii',
  'time': 1630281290670,
  'updated': 1630282292310,
  'tz': None,
  'url': 'https://earthquake.usgs.gov/earthquakes/eventpage/hv72680117',
  'detail': 'https://earthquake.usgs.gov/fdsnws/event/1/query?eventid=hv72680117&format=geojson',
  'felt': None,
  'cdi': None,
  'mmi': None,
  'alert': None,
  'status': 'reviewed',
  'tsunami': 0,
  'sig': 15,
  'net': 'hv',
  'code': '72680117',
  'ids': ',hv72680117,',
  'sources': ',hv,',
  'types': ',origin,phase-data,',
  'nst': 12,
  'dmin': None,
  'rms': 0.07,
  'gap': 80,
  'magType': 'md',
  'type': 'earthquake',
  'title': 'M 1.0 - 8 km SW of Volcano, Hawaii'},
 'geometry': {'type': 'Point', 'coordinates': [-155.281, 19.3831666666667, 1]},
 'id': 'hv72680117'}

### Convert to DataFrame
We need to grab the `properties` section out of every entry in the `features` JSON array to create our dataframe.

In [6]:
earthquake_properties_data = list(
    quake['properties'] 
    for quake in earthquake_json['features']
    )
df = pd.DataFrame(earthquake_properties_data)
df.head()

,mag,place,time,updated,tz,url,detail,felt,cdi,mmi,...,ids,sources,types,nst,dmin,rms,gap,magType,type,title
0,0.99,"8 km SW of Volcano, Hawaii",1630281290670,1630282292310,None,https://earthquake.usgs.gov/earthquakes/eventp...,https://earthquake.usgs.gov/fdsnws/event/1/que...,NaN,NaN,NaN,...,",hv72680117,",",hv,",",origin,phase-data,",12.0,NaN,0.070,80.00,md,earthquake,"M 1.0 - 8 km SW of Volcano, Hawaii"
1,2.69,"7 km ENE of Renfrow, Oklahoma",1630281213530,1630330507392,None,https://earthquake.usgs.gov/earthquakes/eventp...,https://earthquake.usgs.gov/fdsnws/event/1/que...,5.0,3.9,NaN,...,",ok2021qyia,us7000f5ja,",",ok,us,",",dyfi,origin,phase-data,",100.0,0.00000,0.250,63.00,ml,earthquake,"M 2.7 - 7 km ENE of Renfrow, Oklahoma"
2,0.75,"18km ESE of Little Lake, CA",1630281181690,1630330160754,None,https://earthquake.usgs.gov/earthquakes/eventp...,https://earthquake.usgs.gov/fdsnws/event/1/que...,NaN,NaN,NaN,...,",ci40026808,",",ci,",",nearby-cities,origin,phase-data,scitech-link,",10.0,0.06328,0.110,90.00,ml,earthquake,"M 0.8 - 18km ESE of Little Lake, CA"
3,1.22,"14 km SW of Volcano, Hawaii",1630280902320,1630282027660,None,https://earthquake.usgs.gov/earthquakes/eventp...,https://earthquake.usgs.gov/fdsnws/event/1/que...,NaN,NaN,NaN,...,",hv72680097,",",hv,",",origin,phase-data,",19.0,NaN,0.090,92.00,md,earthquake,"M 1.2 - 14 km SW of Volcano, Hawaii"
4,0.80,"0 km N of Walker, California",1630280654134,1630280865462,None,https://earthquake.usgs.gov/earthquakes/eventp...,https://earthquake.usgs.gov/fdsnws/event/1/que...,NaN,NaN,NaN,...,",nn00820498,",",nn,",",origin,phase-data,",8.0,0.02100,0.042,97.25,ml,earthquake,"M 0.8 - 0 km N of Walker, California"


### (Optional) Write Data to CSV

In [7]:
df.to_csv('earthquakes.csv', index=False)

<hr>
<div>
    <a href="./2-creating_dataframes.ipynb">
        <button style="float: left;">&#8592; Previous Notebook</button>
    </a>
    <a href="./4-inspecting_dataframes.ipynb">
        <button style="float: right;">Next Notebook &#8594;</button>
    </a>
</div>
<br>
<hr>